In [1]:
from sweetpea import Factor, MinimumTrials, CrossBlock, synthesize_trials, CMSGen, experiments_to_dicts, tabulate_experiments
from sweetpea import MultiCrossBlock
from sweetpea import ExactlyKInARow
from sweetpea import DerivedLevel
import random

# def trial_items():
#     # generate a randome list with 4 items, 3 letters and 1 number
#     letters = random.sample(['A', 'B', 'C', 'D', 'E'], 3, )
#     number = random.sample(['1', '2', '3', '4'], 1)
#     # combine the lists and shuffle
#     items = letters + number
#     random.shuffle(items)
#     return tuple(items)

def trial_sequences(coherence_ratio:list, motion_direction:list, nr_trials=0):
    coherence_ratio = Factor('coherence_ratio', coherence_ratio)
    motion_direction = Factor('motion_direction', motion_direction)
    item_1 = Factor('Item_1', ['A'])
    item_2 = Factor('Item_2', ['B'])
    item_3 = Factor('Item_3', ['C'])
    item_4 = Factor('Item_4', ['D'])
    item_5 = Factor('Item_5', ['1'])
    item_6 = Factor('Item_6', ['2'])
    item_7 = Factor('Item_7', ['3'])
    item_8 = Factor('Item_8', ['4'])
    choices= Factor('choices', ['NO_KEYS'])
    seq_stimuli = Factor('_num', list(range(1,9)))

    design = [coherence_ratio, motion_direction, item_1,item_2,item_3,item_4,item_5,item_6,item_7,item_8, seq_stimuli, choices]
    crossing = [[coherence_ratio, motion_direction,item_1,item_2,item_3,item_4,item_5,item_6,item_7,item_8, seq_stimuli]]
    constraints = [MinimumTrials(nr_trials)]

    # block = CrossBlock(design, crossing, constraints)
    block = MultiCrossBlock(design, crossing, constraints)

    # block_1 = CrossBlock(design, [], constraints)

    # synthesize trialsequence
    experiment = synthesize_trials(block, 1, CMSGen)

    # export as dict:
    return experiments_to_dicts(block, experiment)

from sweetbean.stimulus import FixationStimulus, BlankStimulus, RandomDotPatternsStimulus
from sweetbean.parameter import TimelineVariable
from sweetbean.sequence import Block    #, Experiment
from sweetbean.sequence import sequence_to_image
from utils import Experiment

def stimulus_sequence(timeline, intensity_1, intensity_2):
    print("SWEETBEAN")
    fixation = FixationStimulus(800)
    blank_1 = BlankStimulus(400)
    blank_2 = BlankStimulus(1000)
    s1 = TimelineVariable('coherence_ratio')
    s2 = TimelineVariable('motion_direction')

    rdp = RandomDotPatternsStimulus(
        duration=2000,
        number_of_oobs=[s1, s2],
        number_of_apertures=2,
        choices=["y", "n"],
    )
    event_sequence = [fixation, blank_1, rdp, blank_2]

    block = Block(event_sequence, timeline)

    experiment = Experiment([block])
    # sequence_to_image(block)
    return experiment.to_html('test_experiment.html')
    # return a js string to transfer to autora
    return experiment.to_js_string(as_function=True, is_async=True)

In [2]:
from sweetbean.parameter import TimelineVariable
from sweetbean.sequence import Block, Experiment
from sweetbean.stimulus import TextStimulus, BlankStimulus, FeedbackStimulus, RandomDotPatternsStimulus, RandomObjectKinematogramStimulus

from utils import rdp_rsvp_stimulus

def stimulus_sequence(timeline, coherence_ratio, movement_direction):
    # introduction TODO write introduction
    introduction = TextStimulus(text='Welcome to this experiment!<br>...<br>Press SPACE to continue.', choices=[" "])

    # instruction TODO write instruction
    instruction = TextStimulus(text='... Press SPACE to continue.', choices=[" "])

    # break TODO incorporate breaks after specific number of trials
    pause = TextStimulus(text='Feel free to take a short break now<br>Press SPACE when you are ready to continue the experiment.', choices=[" "])

    # debriefing/closure TODO write debriefing
    debriefing = TextStimulus(text='... Thank you for your participation!<br>Press SPACE to end the experiment.', choices=[" "])



    # fixation cross and blank screens around it
    fixation_onset = BlankStimulus(duration=680)
    fixation = TextStimulus(duration=915, text="+")
    fixation_offset = BlankStimulus(duration=400)

    # blank screen in between items within a trial
    between_items = BlankStimulus(duration=45)

    # participant response
    response = TextStimulus(text="Which two numbers did you see during the previously displayed sequence?", choices=[1, 2, 3, 4])

    # timeline variables
    # independent variables
    coherence_ratio = TimelineVariable("coherence_ratio", [])
    movement_direction = TimelineVariable("coherence_direction", [])
    item_1 = TimelineVariable('Item_1', [])
    item_2 = TimelineVariable('Item_2', [])
    item_3 = TimelineVariable('Item_3', [])
    item_4 = TimelineVariable('Item_4', [])
    item_5 = TimelineVariable('Item_5', [])
    item_6 = TimelineVariable('Item_6', [])
    item_7 = TimelineVariable('Item_7', [])
    item_8 = TimelineVariable('Item_8', [])
    choices = TimelineVariable('choices', [])

    # rdp = RandomObjectKinematogramStimulus(
    #     duration=750,
    #     number_of_oobs=20,
    #     number_of_apertures=1,
    #     coherence_movement=10,
    #     # coherence_movement=coherence_ratio,
    #     # coherence_orientation=coherence_ratio,
    #     # movement_speed=0.14,
    #     oob_color="white",
    #     background_color="black",
    #     choices=['1','2','3','4']
    # )

    rdp = rdp_rsvp_stimulus(
        duration = 750,
        number_of_oobs=20,
        number_of_apertures=1,
        coherence_movement=10,
        # coherent_movement_direction=movement_direction,
        oob_color="white",
        background_color="black",
        # choices   =choices,
        stimulus_type=1, # 1 is for circles
        text = "item_1",
        prompt="A",
        color="black"
        )
    
    # create all lists of sequences and individual blocks
    introduction_list = [introduction]
    introduction_block = Block(introduction_list)

    instruction_list = [instruction]
    instruction_block = Block(instruction_list)

    training_list = [fixation_onset, fixation, rdp, fixation_offset]
    training_block = Block(training_list)
    experiment_list = [fixation_onset, fixation, rdp, fixation_offset]
    experiment_block = Block(experiment_list, timeline)

    response_list = [response, response]
    respone_block = Block(response_list)

    debriefing_list = [debriefing]
    debriefing_block = Block(debriefing_list)

    # setting up the final experiment consisting of all blocks
    block_list = [introduction_block, instruction_block, training_block, experiment_block, debriefing_block]
    experiment = Experiment(block_list)

    return experiment.to_html('test_experiment.html')
    # return experiment.to_js_string(as_function=True, is_async=True)

In [3]:
trial_seq = trial_sequences([1, 2], [20, 21])
print(len(trial_seq[0]))
# display(trial_seq[0])
# trial_seq[0] = sorted(trial_seq[0], key=int(trial_seq[0]["_num"]))
sorted_trial_seq = sorted(trial_seq[0], key=lambda x: (x['_num'] - 1) % 8)


# Display sorted list
display(sorted_trial_seq)

stimulus_seq = stimulus_sequence(trial_seq[0], 5, 10)
print(stimulus_seq)

Sampling 1 trial sequences using CMSGen.
Encoding experiment constraints...
Running CMSGen...
32


[{'coherence_ratio': 2,
  'motion_direction': 20,
  'Item_1': 'A',
  'Item_2': 'B',
  'Item_3': 'C',
  'Item_4': 'D',
  'Item_5': '1',
  'Item_6': '2',
  'Item_7': '3',
  'Item_8': '4',
  '_num': 1,
  'choices': 'NO_KEYS'},
 {'coherence_ratio': 1,
  'motion_direction': 20,
  'Item_1': 'A',
  'Item_2': 'B',
  'Item_3': 'C',
  'Item_4': 'D',
  'Item_5': '1',
  'Item_6': '2',
  'Item_7': '3',
  'Item_8': '4',
  '_num': 1,
  'choices': 'NO_KEYS'},
 {'coherence_ratio': 2,
  'motion_direction': 21,
  'Item_1': 'A',
  'Item_2': 'B',
  'Item_3': 'C',
  'Item_4': 'D',
  'Item_5': '1',
  'Item_6': '2',
  'Item_7': '3',
  'Item_8': '4',
  '_num': 1,
  'choices': 'NO_KEYS'},
 {'coherence_ratio': 1,
  'motion_direction': 21,
  'Item_1': 'A',
  'Item_2': 'B',
  'Item_3': 'C',
  'Item_4': 'D',
  'Item_5': '1',
  'Item_6': '2',
  'Item_7': '3',
  'Item_8': '4',
  '_num': 1,
  'choices': 'NO_KEYS'},
 {'coherence_ratio': 2,
  'motion_direction': 20,
  'Item_1': 'A',
  'Item_2': 'B',
  'Item_3': 'C',
  '

None
